In [ ]:
import pandas as pd
import os

# DATASET FROM: http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# GF TODO: check if excel present, otherwise download it
# wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

excel_with_path  = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
csv_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.csv'

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster using it than opening excel files\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)

In [ ]:
# define transaction: total cash for a single 'row' of the dataset, i.e. item_price times the number of items bought
df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# day, week and time are added for convenience of analysis later on
from datetime import datetime
df['InvoiceDay'] = df['InvoiceDate'].map(lambda p: p.date())
df['InvoiceWeekDay'] = df['InvoiceDate'].map(lambda p: p.weekday())
df['InvoiceTime'] = df['InvoiceDate'].map(lambda p: p.time())
df['InvoiceWeek'] = df['InvoiceDate'].map(lambda p: p.isocalendar()[1]+52*(p.year-2010))

# avoide negative weeks and start counting from the first week of the dataset, which starts from 01/12/2009
df['InvoiceWeek'] = df['InvoiceWeek']+3


# implement the definition of cancellation in the documentation:
# http://archive.ics.uci.edu/ml/datasets/Online+Retail+II
#      ==> "If this code starts with the letter 'c', it indicates a cancellation.""
def is_cancellation(s):
    if isinstance(s, int):
        #print "ordinary invoice"
        return 0
    elif isinstance(s, unicode):
        #print "unicode string"
        if s.rfind('C')!=-1:
            return 1
        else:
            # print "Something unexpected"  # found, e.g. A506401
            # print s
            return 2
df['IsCancellation'] = df['Invoice'].map(is_cancellation)     
      


# items with prices above ~1500 are so few that is worth looking at them in detail, and excluding them from the plots
# There'a lot of transactions with price set to 0, which based on the descriptions are 
max_item_price=1400
r             =(0,max_item_price)
def is_ordinary_item(p):
    if abs(p)>max_item_price or p==0:
        return 0
    else:
        return 1
df['IsOrdinaryItem'] = df['Price'].map(is_ordinary_item)        

# df.to_csv(csv_with_path, encoding = 'utf-8')

In [ ]:
#df.dtypes

In [ ]:
df.head(5)

In [ ]:
# what are the nulls in each feature?
nulls = df.isnull().sum()[df.isnull().sum() != 0]

# select all transactions containing 
df_nulls = df[df.isnull().any(axis=1)].copy(deep=True)

nulls_rel = nulls/df.shape[0]*100

nulls_summary = pd.concat([nulls, nulls_rel], axis=1, keys=['nulls', 'rel. nulls [%]'])\
               .sort_values('nulls', ascending=False)

nulls_summary

In [ ]:
# df_nulls

In [ ]:
# NEARLY all cancellations have either Quantity or Price set to negative values
# there are only two exceptions, here below:
#           one of which is an actual cancellation
#           the second of which is an onforeseen transaction type which starts with A
(df.loc[ (df.IsCancellation > 0.) & (df.Transaction > 0.) ])

In [ ]:
# there are 6 data entries which are neither normal transactions nor cancellations which are neither 
# they're described as "Adjust bad debt"
(df.loc[ (df.IsCancellation == 2) ])

In [ ]:
# remove the "Adjust bad debt"
df = (df.loc[ (df.IsCancellation < 2) ])

#from IPython.core.display import display, HTML
#display(HTML("""<a href="https://google.at">text</a>"""))

#<a id='the_destination'></a>

## todo
- nan and empty -> cleanup   # IN THE MAKING -> only consequences on the users study
- add colums for data and time, separately ?    # DONE
- create a second DF by user: RFM added to


## What do I want to extract from this dataset?
- https://en.wikipedia.org/wiki/Exploratory_data_analysis

- how many customers   DONE, also as a function of country
- make a pliot of transaction, of purchase prices, and of invoice_price! #  DONE

- VS TIME: trends of spending overall, by country
- revenue as a function of date, as a function of the time in the day  # DONE
- trends of spending: overall, by the largest customer, by the smaller customers (TOO SPECIFIC?)

- NEED TO BUILD A PER CUSTOMER DF
- customer: how many transactions, how much total revenue:
    => BREAK DOWN BY country, tra
- RMF


- ==> do this
- how many different types of items                  # DONE
- what kind of items are bought the most (by NUMBER of by REVENUE), are cancelled the most, 
- how much revenue per type of item bought

- correlation between
- cancellations: fraction of cancellation by nunber of transactions and by proportion of renenue
-                correlation to CHURNING ? Correlation to country OR type of good purchased ?

- cust.groupby('customer_unique_id').size().value_counts() => The majority of customers made only a single purchase. # DONE


- Can I cathegorise the purchasable items? => If so, customers split across those cathegories
- Persona; how many cheap items, few expensive ones?
- https://cxl.com/blog/creating-customer-personas-using-data-driven-research/
- now many items are bought per session, how much is spent per session

==> TOWARDS CUSTOMER PERSONA DEFINITION
- MANY TRANSACTIONS ARE RECORDED AT THE SAME TIME => as if they were a single shopping session, but billed in split goups of goods
- the function summary_data_from_transaction_data treats transactions taking place on the same day as A SINGLE ONE!

- What are the most popular products that are bought in the UK?


---
<a id="section_ID_GF"></a>

- this section can be reached at this URL https://swan001.cern.ch/user/franzoni/notebooks/SWAN_projects/proton/proton-EDA.ipynb#section_ID_GF
- thanks to this documentation

<a id="section_single_items"></a>
## SINGLE ITEM PRICE AND single item purchases
- how many single items are there (ordinary transaction) # DONE
- what is the cost of each single item
- what is the purchase frequency of each item

- what is the resulting price distribution for all transactions

In [ ]:
# I consider here only transactions with positive proceeds

transaction_plus   = (df.loc[df.Transaction > 0.]).Transaction.agg([sum])
transaction_cancel = (df.loc[df.Transaction < 0.]).Transaction.agg([sum])
print('The total cash intake is %e pounds,\n the total amount from cancellations is %e (%f )' \
        %(transaction_plus,transaction_cancel,-100*transaction_cancel/transaction_plus))

In [ ]:
# Exclude cancellations to get the spectrum of single items price
# motivation: the cancellation would give rise to a double counting

# NOTE: cancellations are sometimes negative Quantity for positive price,
# other times they are positive Quantity and negative price
tit='Unit Price'
#(df.loc[ (df.Price > 0.) & (df.Quantity > 0)]).Price.plot(kind='hist',logy=True,title=tit,bins=50,figsize=(7,7))
(df.loc[ (df.IsCancellation ==0 ) ]).Price.plot(kind='hist',logy=True,title=tit,bins=50,figsize=(7,7))

In [ ]:
(df.loc[ (df.IsCancellation ==0 ) & (df['IsOrdinaryItem']==1)]).Price.plot(kind='hist',logy=True,title=tit,bins=50,figsize=(7,7))

In [ ]:
fraction_non_ordinary=100.*len(df.loc[  (df['IsOrdinaryItem']!=1)]) / len( (df.loc[  (df['IsOrdinaryItem']==1)])   )
print('%f percent of the total items are non ordinary, defined as above %d or zero'%(fraction_non_ordinary,max_item_price))

In [ ]:
(df.loc[ (df.IsCancellation ==0 ) ]).Price.plot(kind='hist',logy=True,title=tit,bins=50,range=r,figsize=(7,7))

In [ ]:
(df.loc[ (df.IsCancellation ==0 ) & (df['IsOrdinaryItem']==1) ]).Price.plot(kind='hist',logy=True,title=tit,bins=50,range=r,figsize=(7,7))

In [ ]:
# alternative logic: account for cancelled items, as if they were normal purchases
tit='Unit Price (cancellations included)'
df['Price'].map(lambda p: abs(p)) \
           .plot(kind='hist',logy=True,title=tit,bins=50,range=r,figsize=(7,7))

In [ ]:
# What are the transactions-or-cancellations above max_item_price
# see them all
# df.loc[ (df.Price > max_item_price) | (df.Price < -1*max_item_price) ].head(20000)

# group them by Description to see what they are and how many 
df.loc[ (df.IsOrdinaryItem==0) & (df.Price>0) ].groupby('Description').Description.agg([len]) \
              .rename(columns={"len": "item_huge_price"})            \
              .sort_values(by=['item_huge_price'],ascending=False)


# All prices/cancellatons in excess of 1500 are 'special accounting actions',
# and are not actual goods being purchased

In [ ]:
# how many ORDINARY items are on the cathalogue, based on unique description and price not set to 0
len(df.loc[ df.IsOrdinaryItem==1 ].Description.unique())

In [ ]:
# transactions with 0 prices --> all have odd/dodgy descriptions: manipulation errors and the like
df.loc[ (df.IsOrdinaryItem==1) & (df.Price==0)]
len(df.loc[ (df.IsOrdinaryItem==1) & (df.Price==0)])
# this is 0 by construction, since IsOrdinaryItem flips to 0 of Price==0

In [ ]:
print('Several ordinary items are sold at different prieces; what fraction have different prices and what fraction has identical price?')

def is_price_unique(row):
    if row.min>=row.max and row.min<=row.max:
        row.price_unique=100 
        print('')

        print(row.min)
        print(row.max)
        print(row)
        print('--> the same')
    else:
        row.price_unique=999
        print('')
        print('')
        print(row.min)
        print(row.max)
        print(row)
        print('--> NOT the same')
    return row
# https://stackoverflow.com/questions/27474921/compare-two-columns-using-pandas

#a = df.loc[ df.IsOrdinaryItem==1 ].groupby('Description').Price.agg([len,min,max,sum]) \
#                                  .sort_values(by=['sum'],ascending=False)             \

#b=a.apply(is_price_unique, axis='columns')
#b

# IN THE MAKING
# TO DO: make a spectrum of the price for each element in the cathalogue

# https://stackoverflow.com/questions/17578115/pass-percentiles-to-pandas-agg-function
import numpy as np
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_
    
df_items = df.loc[ df.IsOrdinaryItem==1 ].groupby('Description')        \
                         .Price.agg([len,min,           \
                                    percentile(10),    \
                                    percentile(25),    \
                                    percentile(50),    \
                                    percentile(75),    \
                                    percentile(90),    \
                                    max,sum])          \
                                    .rename(columns={"sum": "item_revenue","len":"item_num"})  

# this is an indication of the spread of the price of a given item
df_items['percentile_7525'] = df_items['percentile_75'] / df_items['percentile_25'] 
df_items['percentile_9010'] = df_items['percentile_90'] / df_items['percentile_10'] 

df_items.loc[ df_items['item_revenue']>10000  ].sort_values(by=['item_revenue'],ascending=False)

In [ ]:
# how much money has been made with each given item
# excluding the top three values which are not purchasable items
tit='total revenue for a given item'
r  = (0,20000)
df_items['item_revenue'].plot(kind='hist',logy=True,title=tit,bins=50,range=r,figsize=(7,7))

In [ ]:
# how many times has each item been sold ?
tit='total num of sales per item'
r  = (0,6000)
df_items['item_num'].plot(kind='hist',logy=True,title=tit,bins=50,range=r,figsize=(7,7))

In [ ]:
# correlation between number of times an item has been sold and its single price ?
df_items.plot.scatter(x='percentile_50',
                      y='item_num',
                      #c='species',
                      #colormap='viridis'
                     )

In [ ]:
df_items['percentile_50'].values

In [ ]:
# this NOT a useful plot
import matplotlib.pyplot as plt
plt.hist2d(df_items['percentile_50'].values,df_items['item_num'].values,
              bins=(10,10),
             #cmap=plt.cm.coolwarm
              cmap=plt.cm.jet
          )

In [ ]:
# relative difference between the 75 and the 25 percentile prices
# as function if item type, ordered by descending per-item total revenue
# the first three items are not actual purchable items
# 
df_items.loc[ (df_items['item_revenue']>7000) & (df_items['percentile_7525']<10) ]                        \
        .sort_values(by=['item_revenue'],ascending=False)[['percentile_7525','percentile_9010']]          \
        .plot.bar(rot=90,figsize=(15,10),logy=False,fontsize=20,ylim=(0,5),subplots=False)

plt.rc('legend',fontsize=20)

<a id="section_transactions"></a>
## Transaction carachteristics

In [ ]:
tit='Transaction'
df['Transaction'].plot(kind='hist',logy=True,title=tit,bins=100,figsize=(15,7))

In [ ]:
from matplotlib.pyplot import *

tit='Revenue per Transaction'
fig, ax = subplots()

df.loc[ (df.Transaction >0) ].Transaction              \
       .plot(kind='hist',logy=True,title=tit,bins=100,  \
             range=(0,10000),figsize=(15,7),colormap='Accent',alpha=0.5,legend=True)

df.loc[ (df.Transaction <0) ].Transaction.map(lambda u: -1*u)              \
       .plot(kind='hist',logy=True,title=tit,bins=100,  \
             range=(0,10000),figsize=(15,7),colormap='autumn',alpha=0.5,legend=True)

ax.legend(['Transactions (positive earnings)', 'Cancellations'])
print('Distribution of the amounts per recorded transaction. Take away message: at costs, the cancellations have larger relative importance')

<a id="section_invoicess"></a>
## Invoices carachteristics
-- can be browsed to directly from here https://nbviewer.jupyter.org/github/franzoni/proton-Hands-on_task_Senior_Data_Scientist/blob/master/proton-EDA.ipynb#section_invoicess

In [ ]:
tit='Invoice'
df.groupby('Invoice').Transaction.agg([sum])    \
   .rename(columns={"sum": "rev_invoice"})            \
   .sort_values(by=['rev_invoice'],ascending=False)  \
   .plot(kind='hist',logy=True,title=tit,bins=100,range=(0,200000),figsize=(15,7))

In [ ]:
tit='Invoice'

fig, ax = subplots()

df_invoices= df.groupby('Invoice').Transaction.agg([sum])    \
   .rename(columns={"sum": "rev_invoice"})            \

df_invoices.loc[ (df_invoices.rev_invoice >0) ]           \
        .sort_values(by=['rev_invoice'],ascending=False).rev_invoice  \
        .plot(kind='hist',logy=True,title=tit,bins=100,range=(0,30000),figsize=(15,7),colormap='Accent',alpha=0.5,legend=True)

df_invoices.loc[ (df_invoices.rev_invoice <0) ]         \
        .sort_values(by=['rev_invoice'],ascending=False).rev_invoice.map(lambda u: -1*u)   \
        .plot(kind='hist',logy=True,title=tit,bins=100,range=(0,30000),figsize=(15,7),colormap='autumn',alpha=0.5,legend=True)

ax.legend(['Invoices (positive earnings)', 'Invoice Cancellations'])
print('Distribution of the amounts per recorded transaction. Take away message: at high costs, the cancellations have larger relative importance')

<a id="section_countries"></a>
## Countries carachteristics

In [ ]:
print('There are %d unique customers'%len(df.Customer_ID.unique()))

In [ ]:
print('Customers are from %d countries'%len(df.Country.unique()))

In [ ]:
# toggle between the whole dataset (df) and the transactions w/ Customer_ID=Nan 
# to check for biases
UU = df

Cust_vs_country = UU.groupby('Country').Customer_ID.unique().agg([len])     \
                    .rename(columns={"len": "num_customers"})               \
                    .sort_values(by=['num_customers'],ascending=False)

# I consider here only transactions with positive proceeds
# I'll make a dedicated plot/column here below for negative/cancelled goods
Revenue_vs_country = (UU.loc[UU.Transaction > 0.])                        \
                     .groupby('Country').Transaction.agg([sum])           \
                    .rename(columns={"sum": "revenue"})                     \
                    .sort_values(by=['revenue'],ascending=False)

Cancellation_vs_country = (UU.loc[UU.Transaction < 0.])                \
                          .groupby('Country').Transaction.agg([sum])    \
                          .rename(columns={"sum": "cancellation"})            \
                          .sort_values(by=['cancellation'],ascending=False)

# I chose to count ANY item listed, be it purchased or cancelled
Items_vs_country = UU.groupby('Country').Invoice.agg([len])    \
                    .rename(columns={"len": "num_items"})            \
                    .sort_values(by=['num_items'],ascending=False)

# I chose to count ANY item listed, be it purchased or cancelled
Invoices_vs_country = UU.groupby('Country').Invoice.unique().agg([len])    \
                    .rename(columns={"len": "num_invoices"})            \
                    .sort_values(by=['num_invoices'],ascending=False)

NoCustomerID_vs_country = df[df.isnull().any(axis=1)]                \
                    .groupby('Country').Transaction.agg([sum])     \
                    .rename(columns={"sum": "no_customer_id"})            \
                    .sort_values(by=['no_customer_id'],ascending=False) \

df_vs_country = Cust_vs_country              \
                .join(Revenue_vs_country )    \
                .join(Cancellation_vs_country)    \
                .join(Items_vs_country)   \
                .join(Invoices_vs_country)   \
                .join(NoCustomerID_vs_country,how='outer')   \
                .fillna(0)                                  \
                .sort_values(by=['revenue'],ascending=False) \

df_vs_country['rel_cancellation']   = \
           -100*df_vs_country['cancellation']/df_vs_country['revenue']

df_vs_country['rel_no_customer_id'] = \
            100*df_vs_country['no_customer_id']/df_vs_country['revenue']

In [ ]:
NoCustomerID_vs_country

In [ ]:
df_vs_country

In [ ]:
# https://www.w3resource.com/pandas/dataframe/dataframe-plot-bar.php
# good reference for plotting directly from pandas (less code!)
df_vs_country.plot.pie(y='num_customers', figsize=(3, 3))

In [ ]:
ax= df_vs_country.drop(['cancellation','no_customer_id'], axis=1)  \
    .plot.bar(rot=90,figsize=(15,25),logy=True,fontsize=20,subplots=True)

ax[0].set(ylabel = 'num. customers')
ax[1].set(ylabel = 'revenue')
ax[2].set(ylabel = 'num. items sold')
ax[3].set(ylabel = 'num. invoices')
ax[4].set(ylabel = 'canc. revenue [%]')
# ax[4].set_yscale('linear')
ax[5].set(ylabel = 'no_cus_id. revenue [%]')
#ax[5].set_yscale('linear')

for v in range(5):
    ax[v].yaxis.get_label().set_fontsize(20)

<a id="section_time"></a>
## Date/Time carachteristics

In [ ]:
df.groupby('InvoiceDay').Transaction.agg(sum).plot(figsize=(25, 3))

In [ ]:
df.groupby('InvoiceWeekDay').Transaction.agg(sum).plot(figsize=(25, 12))
print('Thursday is the day with the largest revenue, Saturday is the least, with near zero!')

In [ ]:
df.groupby('InvoiceTime').Transaction.agg(sum)    \
               .plot(figsize=(25, 3))  # must be limited to >0

In [ ]:
df.groupby('InvoiceWeek').Transaction.agg(sum).plot(figsize=(25, 3))

In [ ]:
# the largest customer by total net revenue (i.e. taking into account also cancellations, in negative)
# is in the UK and has ID 18102.0 and has purchased for 600k pounds
df.groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                  \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Total Customer Revenue'],ascending=False).head(10)        

In [ ]:
# the customer with the largest number of transactions is 
# is in the UK, has ID 17841.0 and has carried out 13.1k transactions
df.groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                  \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Number of Transaction'],ascending=False).head(10)    

In [ ]:
# the customer with the largest number of transactions is 
# is in the UK, has ID 17841.0 and has carried out 12.9 transactions (excluding cancellation transactions)
df.loc[df.Transaction>0]                                                                                      \
    .groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Number of Transaction'],ascending=False).head(10)    

In [ ]:
# the UK generates the largest revenue (16M over the two years),
# surprisingly EIRE is the second largest source of revenue, with 0.6M (despite having only 6 customers, they must be huge customers)
# df.groupby('Country').Transaction.sum().sort_values(ascending=False)
# superseeded by a plot

<a id="section_customers"></a>
## Customers carachteristics

## plan for customers
- make a dataframe
- add also the RFM variables
- add nuber of items bought in total, number of different TYPEs of items
- add total value of course
- Can I cathegorise the purchasable items? => If so, customers split across those cathegories
- Persona; how many cheap items, few expensive ones?
- https://cxl.com/blog/creating-customer-personas-using-data-driven-research/
- now many items are bought per session, how much is spent per session

In [ ]:
df.groupby('Customer_ID').size().value_counts()

<a id='the_destination'></a>

[Pippo goes to the cinema](section-title)